1. Local development 
2. move into GPU EC2 node
3. Add DDP

In [1]:
# TODO: move local environment into ./app folder
# TODO: activate venv somehow
# enable importing python files without caching them
%load_ext autoreload
%autoreload 2

In [2]:
# !pip3 install -r ./app/requirements.txt
!cd app && ./prepare_app_env.py

[prepare_app_env]
[prepare_app_env]:~$ python3 -m venv ../venv-app/
[prepare_app_env]
[prepare_app_env]:~$ . ../venv-app/bin/activate && pip3 install -r ./requirements.txt
[prepare_app_env] Collecting requests==2.31.0 (from -r ./requirements.txt (line 1))
[prepare_app_env]   Obtaining dependency information for requests==2.31.0 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
[prepare_app_env]   Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
[prepare_app_env] Collecting torch==2.0.1 (from -r ./requirements.txt (line 2))
[prepare_app_env]   Using cached torch-2.0.1-cp311-none-macosx_11_0_arm64.whl (55.8 MB)
[prepare_app_env] Collecting torchvision==0.15.2 (from -r ./requirements.txt (line 3))
[prepare_app_env]   Using cached torchvision-0.15.2-cp311-cp311-macosx_11_0_arm64.whl (1.4 MB)
[prepare_app_env] Collecting boto3==1.28.33 (from -r ./requirements.txt (line 4))


In [3]:
# 1. prepare dataset
from app.ml_model.src.download_multinode_dataset import download_dataset

dataset_path = './temp/train_datasets/mnist'
download_dataset(dataset_path)

100.0%


Extracting ./temp/train_datasets/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./temp/train_datasets/mnist/MNIST/raw



100.0%

Extracting ./temp/train_datasets/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./temp/train_datasets/mnist/MNIST/raw




13.9%

100.0%


Extracting ./temp/train_datasets/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./temp/train_datasets/mnist/MNIST/raw



100.0%

Extracting ./temp/train_datasets/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./temp/train_datasets/mnist/MNIST/raw

dataset is downloaded and has 60000 items


In [19]:

# TODO: create temp if not exists???
# throws errors:
# W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs-MacBook-Pro.local, 57625) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
# [W socket.cpp:601] [c10d] The IPv6 network addresses of (Jakubs.....

# 2. run Trainer class via torchrun on my localhost machine
# force  IPv4
!GLOO_SOCKET_IFNAME=eth0 GLOO_SOCKET_FORCE_IPV4=1 torchrun --standalone \
    ./app/ml_model/src/main_mnist_multinode.py \
    --total_epochs 5 \
    --save_every 5 \
    --experiment_name xxxx \
    --dataset_path {dataset_path} \
    --snapshots_dir ../temp/snapshots

NOTE: Redirects are currently not supported in Windows or MacOs.
master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
[GR:0LR:0] is_main_log_node True
[GR:0LR:0] device: cpu
[W socket.cpp:601] [c10d] The IPv6 network addresses of (1.0.0.127.in-addr.arpa, 56653) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (1.0.0.127.in-addr.arpa, 56653) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (1.0.0.127.in-addr.arpa, 56653) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (1.0.0.127.in-addr.arpa, 56653) cannot be retrieved (gai error: 8 - nodename nor servname provided, or not known).
[W socket.cpp:601] [c10d] The IPv6 network addresses of (1.0.0.127.in-addr.arpa, 56653) cannot be retriev

In [8]:

# NOT WORKING as well...
# # TODO: add support for local development? somehow
# !LOCAL_RANK=0 RANK=0 python ./app/ml_model/src/main_mnist_multinode.py \
# --total_epochs 5 \
# --save_every 5 \
# --experiment_name xxxx \
# --dataset_path {dataset_path} \
# --snapshots_dir ../temp/snapshots

[GR:0LR:0] is_main_log_node True
[GR:0LR:0] device: cpu
Traceback (most recent call last):
  File "/Users/jakub.svehla/Desktop/pytorch_parallel_training/apps/app_1/./app/ml_model/src/main_mnist_multinode.py", line 266, in <module>
    main(args.save_every, args.total_epochs, args.batch_size, snapshot_path)
  File "/Users/jakub.svehla/Desktop/pytorch_parallel_training/apps/app_1/./app/ml_model/src/main_mnist_multinode.py", line 225, in main
    ddp_setup()
  File "/Users/jakub.svehla/Desktop/pytorch_parallel_training/apps/app_1/./app/ml_model/src/main_mnist_multinode.py", line 220, in ddp_setup
    init_process_group(backend="gloo")
  File "/Users/jakub.svehla/Desktop/pytorch_parallel_training/.venv/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py", line 900, in init_process_group
    store, rank, world_size = next(rendezvous_iterator)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jakub.svehla/Desktop/pytorch_parallel_training/.venv/lib/python3